#  T-tests and P-values

In statistics, t-test is used to test if two data samples have a significant difference between their means. There are two types of t-test:

* **Student's t-test** (a.k.a. independent or uncorrelated t-test). This type of t-test is to compare the samples of **two independent populations** (e.g. test scores of students in two different classes). `scipy` provides the [`ttest_ind`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_ind.html) method to conduct student's t-test.

* **Paired t-test** (a.k.a. dependent or correlated t-test). This type of t-test is to compare the samples of **the same population** (e.g. scores of different tests of students in the same class). `scipy` provides the [`ttest_rel`](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.ttest_rel.html) method to conduct paired t-test.

Both types of t-tests return a number which is called the **p-value**. If p-value is below 0.05, we can confidently declare the null-hypothesis is rejected and the difference is significant. If p-value is between 0.05 and 0.1, we may also declare the null-hypothesis is rejected but we are not highly confident. If p-value is above 0.1 we do not reject the null-hypothesis.

Read more about the t-test in [this article](http://b.link/test50) and [this Quora](http://b.link/unpaired97). Make sure you understand when to use which type of t-test. 

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

### One tailed t-test 
In a packing plant, a machine packs cartons with jars. It is supposed that a new machine will pack faster on the average than the machine currently used. To test that hypothesis, the times it takes each machine to pack ten cartons are recorded. The results, in seconds, are shown in the tables in the file files_for_lab/ttest_machine.xlsx. Assume that there is sufficient evidence to conduct the t test, does the data provide sufficient evidence to show if one machine is better than the other?

#### Import dataset

In [2]:
# Your code here:
test = pd.read_csv("ttest_machine.txt")
test

,New_machine Old_machine
0,42.1 42.7
1,41 43.6
2,41.3 43.8
3,41.8 43.3
4,42.4 42.5
5,42.8 43.5
6,43.2 43.1
7,42.3 41.7
8,41.8 44
9,42.7 44.1


In [6]:
# First need to split the columns, deliminated by a space 
split = test['New_machine Old_machine'].str.split(' ', expand=True)

# Rename the columns if needed
split.columns = [['new',  'old']]

split

,new,old
0,42.1,42.7
1,41,43.6
2,41.3,43.8
3,41.8,43.3
4,42.4,42.5
5,42.8,43.5
6,43.2,43.1
7,42.3,41.7
8,41.8,44
9,42.7,44.1


In [25]:
split2 = split.apply(pd.to_numeric, errors='coerce')

In [28]:
# Cheating and checking for sanity if there is a difference. There is, but how significant is it the model will tell us
print(split2['new'].mean())
print(split2['old'].mean())

new    42.14
dtype: float64
old    43.23
dtype: float64


In [29]:
# H0 = there is no significant difference in the mean packing time between old machine and new machine (43.2)
# H1 = there is significant difference between old machine and new machine



In [30]:
# Looking at my sample to work out the p value
st.ttest_1samp(split2['new'],9)

TtestResult(statistic=array([153.33539106]), pvalue=array([1.08494877e-16]), df=array([9]))

In [31]:
# The p value is very almost 0 and certainly lower than 0.05, so the no must go
# However, it doesn't seem very reliable with such a small sample size 
# The means seem to be very similar as well, so hard to see how it found them different so confidently

In [32]:
print('statistic :',st.ttest_1samp(split2['new'],9)[0],'; p value (single tailed): ',st.ttest_1samp(split2['new'],9)[1]/2)

statistic : [153.33539106] ; p value (single tailed):  [5.42474384e-17]


#### Import dataset

In this challenge we will work on the Pokemon dataset you have already used. The goal is to test whether different groups of pokemon (e.g. Legendary vs Normal, Generation 1 vs 2, single-type vs dual-type) have different stats (e.g. HP, Attack, Defense, etc.). Use pokemon.csv

In [34]:
# Your code here:
pk = pd.read_csv("pokemon.txt")
pk

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


#### First we want to define a function with which we can test the means of a feature set of two samples. 

In the next cell you'll see the annotations of the Python function that explains what this function does and its arguments and returned value. This type of annotation is called **docstring** which is a convention used among Python developers. The docstring convention allows developers to write consistent tech documentations for their codes so that others can read. It also allows some websites to automatically parse the docstrings and display user-friendly documentations.

Follow the specifications of the docstring and complete the function.

In [37]:
def t_test_features(s1, s2, features=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']):
    """Test means of a feature set of two samples
    
    Args:
        s1 (dataframe): sample 1
        s2 (dataframe): sample 2
        features (list): an array of features to test
    
    Returns:
        dict: a dictionary of t-test scores for each feature where the feature name is the key and the p-value is the value
    """
    results = {}

    for feature in features:
        t_stat, p_value = ttest_ind(s1[feature], s2[feature])
        results[feature] = p_value
    
    return results

In [46]:
from scipy.stats import ttest_ind
sample1 = pk.sample(100)
sample2 = pk.sample(100)
features =['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Total']

In [49]:
t_test_features(sample1, sample2, features)

{'HP': 0.8889730345304673,
 'Attack': 0.799660188791358,
 'Defense': 0.4639625770761644,
 'Sp. Atk': 0.6775746184480705,
 'Sp. Def': 0.7412267626883734,
 'Speed': 0.8695439037711514,
 'Total': 0.6548914951072332}

#### Using the `t_test_features` function, conduct t-test for Lengendary vs non-Legendary pokemons.

*Hint: your output should look like below:*

```
{'HP': 1.0026911708035284e-13,
 'Attack': 2.520372449236646e-16,
 'Defense': 4.8269984949193316e-11,
 'Sp. Atk': 1.5514614112239812e-21,
 'Sp. Def': 2.2949327864052826e-15,
 'Speed': 1.049016311882451e-18,
 'Total': 9.357954335957446e-47}
 ```

In [54]:
# Now to filter by legendary vs non legandary 
sample3 = pk[pk['Legendary'] == True]
sample4 = pk[pk['Legendary'] == False]
t_test_features(sample3, sample4, features)

{'HP': 3.330647684846191e-15,
 'Attack': 7.827253003205333e-24,
 'Defense': 1.5842226094427255e-12,
 'Sp. Atk': 6.314915770427266e-41,
 'Sp. Def': 1.8439809580409594e-26,
 'Speed': 2.3540754436898437e-21,
 'Total': 3.0952457469652825e-52}

#### From the test results above, what conclusion can you make? Do Legendary and non-Legendary pokemons have significantly different stats on each feature?

In [56]:
# Your comment here
# If p is low, no must go. Here, the p values are all incredibly low - this means that they are significantly different 
# and we can look at the mean to double check - and confirm they are much higher in Legendary pokemon
print(sample3['Total'].mean())
print(sample4['Total'].mean())

637.3846153846154
417.21360544217686


#### Next, conduct t-test for Generation 1 and Generation 2 pokemons.

In [59]:
# Your code here
sample5 = pk[pk['Generation'] == 1]
sample6 = pk[pk['Generation'] == 2]
t_test_features(sample5, sample6, features)

{'HP': 0.13791881412813622,
 'Attack': 0.24050968418101445,
 'Defense': 0.5407630349194362,
 'Sp. Atk': 0.14119788176331508,
 'Sp. Def': 0.1678122623160639,
 'Speed': 0.0028356954812578704,
 'Total': 0.5599140649014442}

#### What conclusions can you make?

In [ ]:
# Your comment here
We must accept the null hypothesis - they are not significantly different.  

#### Compare pokemons who have single type vs those having two types.

In [70]:
# Checking the null values and seeing if indeed no type 2 means single type, which it does
pk['Type 2'].value_counts()
pk['Type 2'].isnull().sum()

sample7 = pk[pk['Type 2'].isna()]
sample8 = pk[pk['Type 2'].notna()]
t_test_features(sample7, sample8, features)

{'HP': 0.11060643144431842,
 'Attack': 0.00015741395666164396,
 'Defense': 3.250594205757004e-08,
 'Sp. Atk': 0.0001454917404035147,
 'Sp. Def': 0.00010893304795534396,
 'Speed': 0.02405141079403746,
 'Total': 1.1749035008828752e-07}

#### What conclusions can you make?

In [71]:
# Your comment here
# p is low so no must go, for all except HP which is above the confidence threshold. We can therefore exclude HP and say that's not significantly different

#### Now, we want to compare whether there are significant differences of `Attack` vs `Defense`  and  `Sp. Atk` vs `Sp. Def` of all pokemons. Please write your code below.

*Hint: are you comparing different populations or the same population?*

In [72]:
# Your code here
# I could use my original test code to see if there are any difference in those stats. 
# However, in this set up, we are not comparing attack to defense but rather two populations (from same data set) 
t_test_features(sample1, sample2, features)

{'HP': 0.8889730345304673,
 'Attack': 0.799660188791358,
 'Defense': 0.4639625770761644,
 'Sp. Atk': 0.6775746184480705,
 'Sp. Def': 0.7412267626883734,
 'Speed': 0.8695439037711514,
 'Total': 0.6548914951072332}

In [78]:
# I would likely instead then create a new DF with only attack and defense stats. Then we can compare these two
atk = pk[['Attack','Sp. Atk']]
defc = pk[['Defense','Sp. Def']]

atk.rename(columns={'Attack': '1'}, inplace=True)
atk.rename(columns={'Sp. Atk': '2'}, inplace=True)
defc.rename(columns={'Defense': '1'}, inplace=True)
defc.rename(columns={'Sp. Def': '2'}, inplace=True)

/var/folders/55/2k8v69bx3rb3kj6zvhtb3bcc0000gn/T/ipykernel_48236/2844757950.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atk.rename(columns={'Attack': '1'}, inplace=True)
/var/folders/55/2k8v69bx3rb3kj6zvhtb3bcc0000gn/T/ipykernel_48236/2844757950.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atk.rename(columns={'Sp. Atk': '2'}, inplace=True)
/var/folders/55/2k8v69bx3rb3kj6zvhtb3bcc0000gn/T/ipykernel_48236/2844757950.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

,1,2
0,49,65
1,62,80
2,82,100
3,100,122
4,52,60
...,...,...
795,100,100
796,160,160
797,110,150
798,160,170


In [79]:
# If we now have two populations, one with attack and one with defence, we can say that the attack and defence are significantly different but special attack and defence are not
features2 = ['1', '2']
t_test_features(atk, defc, features2)

{'1': 0.0012123980547321484, '2': 0.5458436328840358}

#### What conclusions can you make?

In [80]:
# Your comment here

# If we now have two populations, one with attack and one with defence, 
# we can say that the attack and defence are significantly different but special attack and defence are not
# p value for attack v defence is 0.0012 but for special it's 0.54
# We can reject the null hypothesis for normal but not for special
